<br>  

## <span style='color:blue'>Section 1: Import</span>  

In [1]:
import pandas as pd
import numpy as np
import copy

from sklearn.model_selection import StratifiedKFold

from sklearn.impute import KNNImputer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from collections import Counter

import statsmodels.api as sm

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

<br>  

## <span style='color:blue'>Section 2: Read, drop columns, form X_train and y_train</span>  

In [2]:
# ---------- read ----------

df_1 = pd.read_csv('../data/code_3_train.csv')

# ---------- drop ----------

print('Before drop :', df_1.shape)
df_1.drop(columns=['index',
                   'pco2', 'ph', 'basophils', 'lactic_acid', 'bmi',
                   'creatine_kinase', 'lymphocyte', 'neutrophils'], inplace=True)
print('After drop:', df_1.shape)
print('')

# ---------- form X ----------

X_train = df_1.drop(columns=['outcome'])
print('X_train :', X_train.shape)

# ---------- form y ----------

y_train = df_1['outcome']
print('y_train :', y_train.shape)
print('')
print('y_train :', np.unique(y_train, return_counts=True))
print('y_train :', Counter(y_train))
print(y_train.value_counts(normalize=True))

Before drop : (882, 51)
After drop: (882, 42)

X_train : (882, 41)
y_train : (882,)

y_train : (array([0., 1.]), array([763, 119], dtype=int64))
y_train : Counter({0.0: 763, 1.0: 119})
0.0    0.865079
1.0    0.134921
Name: outcome, dtype: float64


<br>  

## <span style='color:blue'>Section 3: Function - Impute</span>  

In [3]:
# ---------- this function is to retain the column names of dataframe after missing values imputation ----------

def impute_fit_transform(to_impute, to_impute_fit_transform):
    temp_df = []
    temp_df = to_impute_fit_transform.fit_transform(to_impute)
    temp_df = pd.DataFrame(temp_df, columns=to_impute.columns)
    to_impute = copy.deepcopy(temp_df)
    return to_impute, to_impute_fit_transform

def impute_transform(to_impute, to_impute_fit_transform):
    temp_df = []
    temp_df = to_impute_fit_transform.transform(to_impute)
    temp_df = pd.DataFrame(temp_df, columns=to_impute.columns)
    to_impute = copy.deepcopy(temp_df)
    return to_impute

<br>  

## <span style='color:blue'>Section 4: Function - Scale</span>  

In [4]:
# ---------- this function is to retain the column names of dataframe after scaling ----------

def scale_fit_transform(to_scale, to_scale_fit_transform):
    temp_df = []
    temp_df = to_scale_fit_transform.fit_transform(to_scale)
    temp_df = pd.DataFrame(temp_df, columns=to_scale.columns)
    to_scale = copy.deepcopy(temp_df)
    return to_scale, to_scale_fit_transform

def scale_transform(to_scale, to_scale_fit_transform):
    temp_df = []
    temp_df = to_scale_fit_transform.transform(to_scale)
    temp_df = pd.DataFrame(temp_df, columns=to_scale.columns)
    to_scale = copy.deepcopy(temp_df)
    return to_scale

<br>  

## <span style='color:blue'>Section 5: Manual search and cross validate</span>  

### <span style='color:green'>Reason for manual search:</span>  
* <span style='color:green'>Statsmodels cannot be used in Scikit Learn GridSearchCV and Pipeline.</span>  

In [5]:
# ---------- set 15-fold cross validation ----------

cross_validate = StratifiedKFold(n_splits=15, shuffle=True, random_state=42)

# ---------- set parameters and hyperparameters ----------

search_parameters = {'oversample': [1.00, 0.85, 0.70],
                     'decision_boundary': [0.50, 0.35, 0.20]}

regular_alpha_from = 0
regular_alpha_to = 100
regular_alpha_count = 11

# ---------- initialise result storages ----------

combine_train_validate_accuracy = []
combine_train_validate_recall = []
combine_train_validate_precision = []

train_accuracy = []
train_recall = []
train_precision = []

validate_accuracy = []
validate_recall = []
validate_precision = []

# ---------- loop for oversample SMOTE ----------

for i in range(len(search_parameters['oversample'])):
    
    # ---------- loop for decision boundary ----------
    
    for j in range(len(search_parameters['decision_boundary'])):
        
        # ---------- loop for regularization strength alpha ----------
        
        for alpha_log_reg in np.linspace(regular_alpha_from, regular_alpha_to, regular_alpha_count):
            
            # ---------- loop for cross validation ----------

            for train_index, validate_index in cross_validate.split(X_train, y_train):
    
                # ---------- get train and validate indices ----------
    
                X_train_train, X_train_validate = X_train.iloc[train_index, :], X_train.iloc[validate_index, :]
                y_train_train, y_train_validate = y_train[train_index], y_train[validate_index]
            
                print('Before y_train_train:', Counter(y_train_train))
                print('Before y_train_validate:', Counter(y_train_validate))
                print('Oversample :', search_parameters['oversample'][i])
                print('Decision boundary :', search_parameters['decision_boundary'][j])
                print('alpha =', alpha_log_reg)
                print('')
        
                # ---------- impute on train ----------
    
                knn_impute = KNNImputer(n_neighbors=5)
                X_train_train_impute, knn_impute_fit_transform = impute_fit_transform(X_train_train,
                                                                                      knn_impute)

                # ---------- oversample on train ----------

                o_s = SMOTE(random_state=42,
                            sampling_strategy=search_parameters['oversample'][i])
                X_train_train_impute_over, y_train_train_over = o_s.fit_resample(X_train_train_impute,
                                                                                 y_train_train.ravel())
                
                print('Before_Over y_train_train:', Counter(y_train_train_over))
                print('')

                # ---------- scale on train ----------
    
                ss = StandardScaler()
                X_train_train_impute_over_scale, ss_fit_transform = scale_fit_transform(X_train_train_impute_over,
                                                                                        ss)
    
                # ---------- instantiate and fit regularized on train ----------
    
                X_train_train_impute_over_scale = sm.add_constant(X_train_train_impute_over_scale)
                lr = sm.Logit(y_train_train_over, X_train_train_impute_over_scale)
                lr_result = lr.fit_regularized(maxiter=500, method='l1', alpha=alpha_log_reg)
    
                # ---------- impute and scale on train ----------
                # ---------- not using oversample to have fair comparison to validate performance ----------

                X_train_train_impute = impute_transform(X_train_train,
                                                        knn_impute_fit_transform)
                X_train_train_impute_scale = scale_transform(X_train_train_impute,
                                                             ss_fit_transform)

                # ---------- predict and evaluate on train ----------

                X_train_train_impute_scale = sm.add_constant(X_train_train_impute_scale)
                y_train_train_predicted = (lr_result.predict(X_train_train_impute_scale)>=search_parameters['decision_boundary'][j]).astype(int)
        
                train_accuracy.append(accuracy_score(y_train_train, y_train_train_predicted))
                train_recall.append(recall_score(y_train_train, y_train_train_predicted))
                train_precision.append(precision_score(y_train_train, y_train_train_predicted))
    
                # ---------- impute and scale on validate ----------
    
                X_train_validate_impute = impute_transform(X_train_validate,
                                                           knn_impute_fit_transform)
    
                X_train_validate_impute_scale = scale_transform(X_train_validate_impute,
                                                                ss_fit_transform)
    
                # ---------- predict and evaluate on validate ----------
    
                X_train_validate_impute_scale = sm.add_constant(X_train_validate_impute_scale)
                y_train_validate_predicted = (lr_result.predict(X_train_validate_impute_scale)>=search_parameters['decision_boundary'][j]).astype(int)
        
                validate_accuracy.append(accuracy_score(y_train_validate, y_train_validate_predicted))
                validate_recall.append(recall_score(y_train_validate, y_train_validate_predicted))
                validate_precision.append(precision_score(y_train_validate, y_train_validate_predicted))
    
            combine_train_validate_accuracy.append([search_parameters['oversample'][i],
                                                    search_parameters['decision_boundary'][j],
                                                    alpha_log_reg,
                                                    np.mean(train_accuracy),
                                                    np.std(train_accuracy),
                                                    np.mean(validate_accuracy),
                                                    np.std(validate_accuracy)])
            combine_train_validate_recall.append([search_parameters['oversample'][i],
                                                  search_parameters['decision_boundary'][j],
                                                  alpha_log_reg,
                                                  np.mean(train_recall),
                                                  np.std(train_recall),
                                                  np.mean(validate_recall),
                                                  np.std(validate_recall)])
            combine_train_validate_precision.append([search_parameters['oversample'][i],
                                                     search_parameters['decision_boundary'][j],
                                                     alpha_log_reg,
                                                     np.mean(train_precision),
                                                     np.std(train_precision),
                                                     np.mean(validate_precision),
                                                     np.std(validate_precision)])
            
            train_accuracy = []
            train_recall = []
            train_precision = []

            validate_accuracy = []
            validate_recall = []
            validate_precision = []

Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 0.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.38177929886068407
            Iterations: 176
            Function evaluations: 176
            Gradient evaluations: 176
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 0.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.3971100663783948
            Iterations: 177
            Function evaluations: 177
            Gradient evaluations: 177
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.43745865376509097
            Iterations: 60
            Function evaluations: 61
            Gradient evaluations: 60
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 10.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4428146706306205
            Iterations: 64
            Function evaluations: 64
            Gradient evaluations: 64
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 10.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.4659407010678905
            Iterations: 74
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5057521243086694
            Iterations: 58
            Function evaluations: 58
            Gradient evaluations: 58
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 20.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.503016818738303
            Iterations: 58
            Function evaluations: 59
            Gradient evaluations: 58
Before y_train_train: Counter({0.0: 713, 1.0: 111})
Before y_train_validate: Counter({0.0: 50, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 20.0

Before_Over y_train_train: Counter({1.0: 713, 0.0: 713})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5021775082727546
            Iterations: 58
           

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5514843072454776
            Iterations: 42
            Function evaluations: 42
            Gradient evaluations: 42
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 40.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5604877140374724
            Iterations: 46
            Function evaluations: 46
            Gradient evaluations: 46
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 40.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5749617570839598
            Iterations: 46
          

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5695467297674939
            Iterations: 40
            Function evaluations: 41
            Gradient evaluations: 40
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 50.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5896301076256792
            Iterations: 41
            Function evaluations: 42
            Gradient evaluations: 41
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 1.0
Decision boundary : 0.5
alpha = 50.0

Before_Over y_train_train: Counter({1.0: 712, 0.0: 712})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5837257925482628
            Iterations: 37
          

...

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5783538978146087
            Iterations: 41
            Function evaluations: 42
            Gradient evaluations: 41
Before y_train_train: Counter({0.0: 713, 1.0: 111})
Before y_train_validate: Counter({0.0: 50, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 40.0

Before_Over y_train_train: Counter({0.0: 713, 1.0: 499})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5727776360600966
            Iterations: 45
            Function evaluations: 46
            Gradient evaluations: 45
Before y_train_train: Counter({0.0: 712, 1.0: 112})
Before y_train_validate: Counter({0.0: 51, 1.0: 7})
Oversample : 0.7
Decision boundary : 0.2
alpha = 40.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.5801074847419185
            Iterations: 40
          

Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 60.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6079450907673574
            Iterations: 37
            Function evaluations: 37
            Gradient evaluations: 37
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 60.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.609485914592101
            Iterations: 37
            Function evaluations: 37
            Gradient evaluations: 37
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
al

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6304341231388246
            Iterations: 36
            Function evaluations: 37
            Gradient evaluations: 36
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 70.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6473286561395846
            Iterations: 35
            Function evaluations: 35
            Gradient evaluations: 35
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 70.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6517715565355284
            Iterations: 30
            Function evaluations: 31
            Gradient evaluations: 30
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 90.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6546380462306467
            Iterations: 29
            Function evaluations: 30
            Gradient evaluations: 29
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 90.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6629966893802018
            Iterations: 29
          

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6651956136922024
            Iterations: 28
            Function evaluations: 28
            Gradient evaluations: 28
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 100.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6721717906512032
            Iterations: 25
            Function evaluations: 26
            Gradient evaluations: 25
Before y_train_train: Counter({0.0: 712, 1.0: 111})
Before y_train_validate: Counter({0.0: 51, 1.0: 8})
Oversample : 0.7
Decision boundary : 0.2
alpha = 100.0

Before_Over y_train_train: Counter({0.0: 712, 1.0: 498})

Optimization terminated successfully    (Exit mode 0)
            Current function val

<br>  

## <span style='color:blue'>Section 6: Save results</span>  

In [6]:
combine_train_validate_accuracy = pd.DataFrame(combine_train_validate_accuracy,
                                               columns=['oversample',
                                                        'decision_boundary',
                                                        'alpha',
                                                        'train_accuracy_mean',
                                                        'train_accuracy_std_dev',
                                                        'validate_accuracy_mean',
                                                        'validate_accuracy_std_dev'])
combine_train_validate_accuracy['overfit'] = (combine_train_validate_accuracy['validate_accuracy_mean']-combine_train_validate_accuracy['train_accuracy_mean'])/combine_train_validate_accuracy['train_accuracy_mean']*100
combine_train_validate_accuracy.to_csv('../data/code_4_logreg_regz_train_validate_accuracy.csv', na_rep='NaN', index_label='index')

In [7]:
combine_train_validate_recall = pd.DataFrame(combine_train_validate_recall,
                                             columns=['oversample',
                                                      'decision_boundary',
                                                      'alpha',
                                                      'train_recall_mean',
                                                      'train_recall_std_dev',
                                                      'validate_recall_mean',
                                                      'validate_recall_std_dev'])
combine_train_validate_recall['overfit'] = (combine_train_validate_recall['validate_recall_mean']-combine_train_validate_recall['train_recall_mean'])/combine_train_validate_recall['train_recall_mean']*100
combine_train_validate_recall.to_csv('../data/code_4_logreg_regz_train_validate_recall.csv', na_rep='NaN', index_label='index')

In [8]:
combine_train_validate_precision = pd.DataFrame(combine_train_validate_precision,
                                                columns=['oversample',
                                                         'decision_boundary',
                                                         'alpha',
                                                         'train_precision_mean',
                                                         'train_precision_std_dev',
                                                         'validate_precision_mean',
                                                         'validate_precision_std_dev'])
combine_train_validate_precision['overfit'] = (combine_train_validate_precision['validate_precision_mean']-combine_train_validate_precision['train_precision_mean'])/combine_train_validate_precision['train_precision_mean']*100
combine_train_validate_precision.to_csv('../data/code_4_logreg_regz_train_validate_precision.csv', na_rep='NaN', index_label='index')

### <span style='color:green'>Refer to slides for visualisation of analysis</span>  

3 Jan 2022